# Тестирование API

Данный ноутбук предназначен для тестирования API-сервиса. В случае каких либо изменений, должны быть пройдены тесты из этого ноутбука для проверки базовой работоспособности сервиса.

Проводимые тесты:

### POST методы:

**1) /fit_request - обучение модели путем подачи данных напрямую JSON запросом. Шаблон:**

```json
{
    "X": [[1.0, 2.0],[3.0, 4.0],[5.0, 6.0]],
    "y": [0,1,0],
    "config": {
        "id": "model_name",
        "ml_model_type": "logistic",
        "hyperparameters": {
            "solver": "liblinear",
            "max_iter": 100
        }
    }
}
```
> Гиперпараметры `solver` и `max_iter` приведены как пример, их использование не обязательное

**2) /fit_csv - обучение модели путем подачи данных в виде CSV файла. Пример CURL запроса:**

```bash
curl -X POST "http://127.0.0.1:8000/api/v1/models/fit_csv" \
-F "file=@data.csv" \
-F "target_column=target" \
-F 'config={
  "id": "logistic_model_csv",
  "ml_model_type": "logistic",
  "hyperparameters": {
    "solver": "liblinear",
    "max_iter": 100
  }
}'
```

**3) /predict_request - получение предсказаний путем подачи данных, аналогичным способу из теста /fit_response. Шаблон:**

```json
[
    {
        "id": "logistic_model_json",
        "X": [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
    }
]
```

**4) /predict_csv - получение предсказаний путем подачи данных, аналогичным способу из теста /fit_csv. Пример CURL запроса:**

```bash
curl -X POST "http://127.0.0.1:8000/api/v1/models/predict_csv" \
-F "file=@data.csv" \
-F "model_id=logistic_model_csv"
```

### GET запросы:

**5) /list_models - возвращает список всех имеющихся обученных моделей в сервисе, вместе с их типом и метриками, полученными при обучении. Пример CURL запроса:**

```bash
curl -X GET "http://127.0.0.1:8000/api/v1/models/list_models"
```

### DELETE запросы:

**6) /remove/{model_id} - по запросу удаляет из хранилища сервиса и регистра модель с определенным идентификатором. Пример CURL запроса:**

```bash
curl -X DELETE "http://127.0.0.1:8000/api/v1/models/remove/some_model"
```

**7) /remove_all - по запросу удаляет из хранилища сервиса и регистра все имеющиеся модели (Использовать обдуманно!). Пример CURL запроса:**

```bash
curl -X DELETE "http://127.0.0.1:8000/api/v1/models/remove_all"
```

Само тестирование вы можете провести прожав ячейки ниже.

### Блок импорта модулей, настройки базовых переменных и используемых функций

In [1]:
import requests
import pandas as pd
import json
import numpy as np
from IPython.display import display, HTML

BASE_URL = "http://127.0.0.1:8000/api/v1/models"

# Утилита для отображения результатов в виде HTML
def display_response(endpoint, response):
    html_content = f"""
    <div style="border: 2px solid #4CAF50; padding: 10px; margin: 10px; border-radius: 5px;">
        <h3 style="color: #4CAF50;">Endpoint: {endpoint}</h3>
        <pre>{json.dumps(response, indent=4)}</pre>
    </div>
    """
    display(HTML(html_content))

# Генерация данных для /fit_request и /predict_request
X_data = np.random.rand(1000, 15).tolist()  # 1000 строк, 15 фичей
y_data = np.random.randint(0, 2, size=1000).tolist()  # Целевые переменные (0 или 1)

# Создаем CSV файл для /fit_csv и /predict_csv
df = pd.DataFrame(X_data, columns=[f"feature_{i}" for i in range(1, 16)])
df["target"] = y_data
csv_file_path = "test_dataset_large.csv"
df.to_csv(csv_file_path, index=False)

### 1. Тестирование /fit_request

In [2]:
fit_request_data = {
    "X": X_data,
    "y": y_data,
    "config": {
        "id": "test_model_json_large",
        "ml_model_type": "logistic",
        "hyperparameters": {
            "solver": "liblinear",
            "max_iter": 100
        }
    }
}

response = requests.post(f"{BASE_URL}/fit_request", json=fit_request_data)
display_response("/fit_request", response.json())

### 2. Тестирование /fit_csv

In [3]:
with open(csv_file_path, "rb") as csv_file:
    files = {"file": csv_file}
    data = {
        "target_column": "target",
        "config": json.dumps({
            "id": "test_model_csv_large",
            "ml_model_type": "logistic",
            "hyperparameters": {
                "solver": "liblinear",
                "max_iter": 100
            }
        })
    }
    response = requests.post(f"{BASE_URL}/fit_csv", data=data, files=files)
    display_response("/fit_csv", response.json())

### 3. Тестирование /predict_request

In [4]:
predict_request_data = [
    {
        "id": "test_model_json_large",
        "X": np.random.rand(10, 15).tolist()  # Используем 10 строк для предсказаний
    }
]
response = requests.post(f"{BASE_URL}/predict_request", json=predict_request_data)
display_response("/predict_request", response.json())

### 4. Тестирование /predict_csv

In [5]:
# Создаем новый CSV без целевой колонки "target" для предсказаний
predict_csv_file_path = "test_dataset_large_predict.csv"
df_predict = df.drop(columns=["target"])  # Удаляем колонку target
df_predict.to_csv(predict_csv_file_path, index=False)

with open(predict_csv_file_path, "rb") as csv_file:
    files = {"file": csv_file}
    data = {"model_id": "test_model_csv_large"}
    response = requests.post(f"{BASE_URL}/predict_csv", data=data, files=files)
    display_response("/predict_csv", response.json())

### 5. Тестирование /list_models

In [6]:
response = requests.get(f"{BASE_URL}/list_models")
display_response("/list_models", response.json())

### 6. Тестирование /remove/{model_id}

In [7]:
model_id = "test_model_csv_large"
response = requests.delete(f"{BASE_URL}/remove/{model_id}")
display_response(f"/remove/{model_id}", response.json())

### 7. Тестирование /remove_all

In [8]:
response = requests.delete(f"{BASE_URL}/remove_all")
display_response("/remove_all", response.json())